 # Practice  Soft SVM.
 ----
  
  University : UTEC \\
  Course       : Machine Learning \\
  Professor    : Cristian López Del Alamo \\
  Topic        : Soft SVM \\
  Termina      : 4:45
   

 ----

Write the names and surnames of the members and the percentage of participation of each one in the development of the practice:
 - Integrante 1: (%)
 - Integrante 2: (%)
 - Integrante 3: (%)
 - Integrante 4: (%)


 ----


  




In [4]:
# Loading Libraries
# %pip install pywavelets
import numpy as np
import math
import pandas as pd
import random
import matplotlib.pyplot as plt
import cvxopt
import seaborn as sns
import itertools
import pywt
import os
from sklearn.metrics import confusion_matrix
import zipfile
from skimage import io, color

## Normalization

In [5]:
# Data Normalizing
def normalizacion(data):
  """
  Normaliza los datos para que tengan una media de 0 y una desviación estándar de 1.
  
  Parameters:
  data (numpy.ndarray): Un array de numpy con los datos a normalizar.
  
  Returns:
  numpy.ndarray: Los datos normalizados.
  """
  mean = np.mean(data, axis=0)
  std = np.std(data, axis=0)
  normalized_data = (data - mean) / std
  return normalized_data

## 1. Feature Vector

A wavelet is a more efficient method than Fourier transforms for approximating a non-periodic function. Since an image can be seen as a two-dimensional function in a 3D space, we can approximate it using wavelets. In this case, we use the Hard Wavelet. The goal is to obtain a characteristic vector that represents each image

[library:](https://pywavelets.readthedocs.io/en/latest/index.html)
----
Parámetros de la Función **haar**
- f: Image File Name
- n: Number of slider in the image

In [6]:
# Wavelet function for extracting image characteristics. This function returns a vector with image characteristics
def haar(f, n):
  imagen = io.imread(f)
  for i in range(n):
    imagen, (LH, HL, HH) = pywt.dwt2(imagen, 'haar')
  gfg = np.array(imagen)
  return gfg.flatten()

### Encode Function
Encode is a function that encodes images into feature vectors using wavelets to extract relevant information from each image. Encode takes the path where the images are located, encodes them, and returns a matrix where the ith element of the matrix is the feature vector of image i.

In [7]:
# This function loads all images in the specified path and returns a vector containing the feature vectors of these images. In the example, we use three splits.
def encode(path, cuts):
  encodings = []

  print("Ingresó : ")
  print(path)
  i=0
  for filename in os.listdir(path):
      image_file = os.path.join(path, filename)
      print(image_file)
      if os.path.isfile(image_file):
          encodings.append(haar(image_file, cuts))
      i=i+1
      if i==20:
        return encodings
  return encodings

## Loading the data and extracting features using Hard Wavelet

### get_data Function
This function takes as input the paths where images of happy and disgust people are located, encodes them, and stores them in matrices. One matrix contains rows with features of happy people and another matrix contains features of sad people; both are combined into the same matrix. The rows of the matrix are randomized to then create the database that will train our soft SVM

In [8]:

def get_data(emotion_1, emotion_2, cuts):
  data  =  np.array(encode(emotion_1,cuts))
  data  = np.insert(data, 0, 1, axis=1)
  temp =  np.array(encode(emotion_2,cuts))
  temp = np.insert(temp, 0, -1, axis=1)
  data = np.concatenate((data, temp), axis=0)
  for i in range(10):
    np.random.shuffle(data)
  y = data[:,0]
  x = data[:, 1:]
  return x, y

# Load train and test data

We mount the drive in Colab, get the path where our images are located, and load the test and training data

[Download DataSet](https://drive.google.com/drive/folders/1TvgyPE5l6TWtx7Fa6ZSKVUG2SsvEvh7l?usp=sharing)

In [27]:
# from google.colab import drive
# drive.mount('/content/drive')

# #  In the variable path, write the path where your dataset is located
# path = '/content/drive/MyDrive/UTEC/CURSOS/2022.2/Inteligencia Artificial/DataSet/images'
print("path:", os.getcwd())
path = os.getcwd() + "/images"

c1_emotion = 'fear'
c2_emotion = 'neutral'

c1_emotion_train  = path +  '/train/' + c1_emotion
c2_emotion_train  = path +  '/train/' + c2_emotion

c1_emotion_test  = path +  '/validation/' + c1_emotion
c2_emotion_test  = path +  '/validation/' + c2_emotion

train_x, train_y = get_data(c1_emotion_train, c2_emotion_train,1)
test_x, test_y = get_data(c1_emotion_test, c2_emotion_test,1)

path: /home/jamcy/Github/2024-II/ML/W5
Ingresó : 
/home/jamcy/Github/2024-II/ML/W5/images/train/fear
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/7224.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/3717.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/13654.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/11767.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/12430.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/4891.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/32706.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/9227.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/23597.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/29436.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/110.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/14013.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/23830.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/22257.jpg
/home/jamcy/Github/2024-II/ML/W5/images/train/fear/

In [28]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(40, 576)
(40,)
(40, 576)
(40,)


### 1. Hypothesis :  Remember, you can achieve it using linear algebra.

$h(x_i) = x_i*w^t + b$

In [60]:
def h(X,w,b):
  """
  Hipótesis de la SVM

  Parameters:
  X -- Matriz de datos de tamaño (m, n)
  w -- Vector de pesos de tamaño (1, n)
  b -- Bias

  Returns: 
  numpy.ndarray -- Vector de tamaño (m, 1) con las predicciones de la SVM
  """
  return np.dot(X, w) + b

### 2. Loss function

$L = \frac{1}{2}\parallel W \parallel_2^2  + C\sum_{i=0}^nmax(0,1-y_i(x_iw^t + b)) $


In [37]:
# Implemente la función de pérdida del soft SVM
def loss(y,y_aprox,W,C):
  """
  Función de pérdida de la SVM

  Parameters:
  y -- Vector de etiquetas de tamaño (m, 1)
  y_aprox -- Vector de predicciones de tamaño (m, 1)
  W -- Vector de pesos de tamaño (1, n)
  C -- Parámetro de regularización

  Returns:
  float -- Valor de la función de pérdida
  """
  
  #Regularization  first  2 items
  regularization_term = 1/2 * (np.linalg.norm(W)**2)

  #Loss Term
  h_loss = np.maximum(0, 1 - y * y_aprox)
  h_loss_sum = C * np.sum(h_loss)

  #Finding the total loss
  total_loss = regularization_term + h_loss_sum
  return total_loss

### 3. Derivatives

1. iF ($y_i*h(x_i)) \lt 1 $
 - $\frac{\partial L}{\partial w} = w + C\sum_{
 i=0}^n-y_i*x_i $
2. else
 - $\frac{\partial L}{\partial w} = w $

----
**Don't forget to find the derivatives for Bias.**







In [51]:
# Implemente la función para obtener las derivadas de W
def derivatives(x, y,y_aprox, w, b, C):
  """
  Derivadas de la función de pérdida de la SVM

  Parameters:
  x -- Matriz de datos de tamaño (m, n)
  y -- Vector de etiquetas de tamaño (m, 1)
  y_aprox -- Vector de predicciones de tamaño (m, 1)
  w -- Vector de pesos de tamaño (1, n)
  b -- Bias
  C -- Parámetro de regularización

  Returns:
  numpy.ndarray -- Vector de tamaño (1, n) con las derivadas de W
  """
  
  # Condición para cada elemento
  condition = (y * y_aprox) < 1
  
  # Derivadas de w y b
  dw = np.where(condition[:, np.newaxis], w + C * (-y[:, np.newaxis] * x), w)
  db = np.where(condition, -C * y, 0)
  
  # Sumar las derivadas a lo largo de las filas
  dw = np.sum(dw, axis=0)
  db = np.sum(db, axis=0)
  
  return db, dw

### 4. Change Parameters


1. iF ($y_i*h(x_i)) \lt 1 $
 - $w  = w -  \alpha( C\sum_{
 i=0}^n-y_i*x_i) $
2. else
 - $w = w -  \alpha*w$

----
**Don't forget charge the Bias.**


In [56]:
def Update(x,y,y_aprox,w, b, db, dw, alpha):
  """
  Función para actualizar los pesos de la SVM

  Parameters:
  x -- Matriz de datos de tamaño (m, n)
  y -- Vector de etiquetas de tamaño (m, 1)
  y_aprox -- Vector de predicciones de tamaño (m, 1)
  w -- Vector de pesos de tamaño (1, n)
  b -- Bias
  db -- Derivada de b
  dw -- Derivada de w
  alpha -- Tasa de aprendizaje

  Returns:
  w -- Vector de tamaño (1, n) con los pesos actualizados
  b -- Bias actualizado
  """

  # Condición para cada elemento
  condition = (y * y_aprox) < 1
  
  # Actualización de w y b
  w = np.where(condition[:, np.newaxis], w - alpha * dw, w - alpha * w)
  b = np.where(condition, b - alpha * db, b)

  return w, b


# 5. Training stage

In [58]:
def training(X, Y, C, alpha, epochs):
  w = np.array([np.random.rand() for i in range(X.shape[1])])
  b = np.random.rand()
  error = []
  for i in range(epochs):
    Y_aprox = h(X,w,b)
    db, dw = derivatives(X, Y,Y_aprox, w, b, C)
    w, b = Update(X,Y,Y_aprox,w, b, db, dw, alpha)
    L = loss(Y,Y_aprox,w,C)
    error.append(L)
  return w, b, error

### 6. Testing Stage

In [54]:
# Implemente la función de testing
def testing(X,W,b):
  y_aprox = []
  # write your code here
  for i in range(X.shape[0]):
    y_aprox = np.sign(h(X[i], W, b))
  return np.array(y_aprox)

In [69]:
# Main Program

m = train_y.size
k = train_x[0].size
train_x_norm = np.apply_along_axis(normalizacion, 1, train_x)
test_x_norm = np.apply_along_axis(normalizacion, 1, test_x)

W, b, e1, = training(train_x_norm, train_y, 1e6, 1e-8, 1200)
m = test_y.size
y_pred = testing(test_x_norm, W, b)
test_y = test_y.astype('int')

print("Clasificados correctamente:", correct)
print("Clasificados incorrectamente:", len(test_y)-correct)
print("% de efectividad", round(100*correct/len(test_y), 2))


ValueError: shapes (40,576) and (40,576) not aligned: 576 (dim 1) != 40 (dim 0)

In [ ]:
matrix = confusion_matrix(test_y,y_pred)
df2 = pd.DataFrame(matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis], index=["Angry", 'Happy'], columns=["Angry", 'Happy'])
sns.heatmap(df2, annot=True, cbar=None, cmap="Greens")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.xlabel("Predicted")
plt.ylabel("Real")
plt.show()

[Documento de excel](https://docs.google.com/spreadsheets/d/1yxYCjj_uS2Wrj8ofAc7hiRoCYX0-wJNiTMzGnWdsZoA/edit?usp=sharing)